# Backtest Optimization with NVIDIA RAPIDS
### This notebook showcases <b>[RAPIDS](https://developer.nvidia.com/rapids)</b> providing over 
### * <b>+700%</b> performance boost
### * <b>~40K</b> backtests on moving average buy/sell indicators
### * <b>All</b> stocks in the <b>S&P500</b>.

---

In [2]:
# !pip install kaggle plotly

---

# Data - Kaggle Stocks  
For simplicity, we userecent [stock market data from Kaggle](https://www.kaggle.com/datasets/andrewmvd/sp-500-stocks).  
This data set is updated regularly.  This assumes you have a Kaggle account and have create a key.

In [5]:
# import os
# os.environ['KAGGLE_USERNAME'] = "xxxxxxxx"
# os.environ['KAGGLE_KEY'] = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
!rm -rf sp-500-stocks.zip sp500_*.csv
!kaggle datasets download -d andrewmvd/sp-500-stocks
!unzip sp-500-stocks.zip

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/sp-500-stocks
License(s): CC0-1.0
 92%|███████████████████████████████████▏  | 41.0M/44.3M [00:00<00:00, 46.7MB/s]
100%|██████████████████████████████████████| 44.3M/44.3M [00:00<00:00, 48.6MB/s]
Archive:  sp-500-stocks.zip
  inflating: sp500_companies.csv     
  inflating: sp500_index.csv         
  inflating: sp500_stocks.csv        


---
# Sample Backtest
##### This backtest sample is the mean price of the S&P.  A Moving Average (MA) or Simple Moving Average (SMA) using a Window Size of 50 is used to create buy/sell signals.  When the current price is below the 50 Day MA, a buy is triggered and when the price moves above the 50 Day MA, a sell is triggered.  A negative buy and sell was triggered on May 1 and May 7, respectively.  This caused a Max Drawdown (MDD) in the account.

In [6]:
from plotly import graph_objects as go
import pandas as pd
import numpy as np
import plotly.io as pio
pio.renderers.default = 'iframe'

win_sz = 50

sp500 = pd.read_csv("sp500_stocks.csv")#.set_index("Date", drop=True)
sp500 = sp500.drop("Symbol", axis=1)
sp500.columns = [col.lower() for col in sp500.columns] 
sp500 = sp500.tail(252*2).copy()
sp500 = sp500.groupby('date').mean().reset_index(drop=False)
display(sp500.sample(3))

sp500["ma"] = sp500["close"].rolling(window=win_sz, min_periods=0).mean().reset_index(0,drop=True)

sp500["signal"] =  ( (sp500["close"] < sp500["ma"]).astype(int)* -1 + (sp500["close"] > sp500["ma"]).astype(int)  ) 
sp500["signal"]  = (sp500["signal"] != sp500["signal"].shift(1)).astype(int) * sp500["signal"]
sp500.loc[0, "signal"] = 0

last_buy_price = sp500.loc[sp500["signal"] == -1].copy()["close"].rename("last_buy_price")
sp500 = sp500.merge(last_buy_price, left_index=True, right_index=True, how="left").fillna(0)
sp500["last_buy_price"] = sp500["last_buy_price"].replace(0.0, np.nan).ffill().fillna(0)

sell_mask = (sp500["signal"] == 1) & (sp500["last_buy_price"] > 0)
sp500.loc[sell_mask, "profit_pct_plus_1"] = 1 + ( (sp500["close"] - sp500["last_buy_price"]) / sp500["last_buy_price"] )
sp500["profit_pct_plus_1"] = sp500["profit_pct_plus_1"].fillna(1.0)

sp500["pct_cumprod"] = sp500["profit_pct_plus_1"].cumprod()
sp500["pct_cummax"] = sp500["pct_cumprod"].cummax()

sp500["drawdown"] = sp500["pct_cumprod"] - sp500["pct_cummax"]
sp500["max_drawdown"] = sp500["drawdown"].cummin()

roi = sp500.iloc[-1]["pct_cumprod"] - 1
roi = roi.round(4) * 100

stock_plot = go.Ohlc(x=sp500['date'], open=sp500['open'], high=sp500['high'], low=sp500['low'], close=sp500['close'], name="SP500 Mean", increasing_line_color= 'cyan', decreasing_line_color= 'gray')

fig = go.Figure(data=[stock_plot])

# MA
fig.add_trace(go.Scatter(x=sp500["date"], y=sp500["ma"], name=f"MA {win_sz}", opacity=0.7, line=dict(width=2, color="black")))

marker_size=10
marker_border=1

# BUY
buy_mask = sp500["signal"] == -1
purchases = sp500.loc[buy_mask].copy().reset_index(drop=True)
fig.add_trace(go.Scatter(
    x=purchases["date"], y=purchases["close"], mode="markers", name="buy", 
    marker=dict(symbol="star-triangle-up", size=marker_size, color='blue', line=dict(width=marker_border, color="DarkSlateGrey"))))

# SELL
sales = sp500.loc[sp500["signal"] == 1].copy().reset_index(drop=True)
fig.add_trace(go.Scatter(
    x=sales["date"], y=sales["close"], mode="markers", name="sell",
    marker=dict(symbol="star-triangle-down", size=marker_size,  color='green', line=dict(width=marker_border, color="DarkSlateGrey"))))

# MDD
mdd = sp500["max_drawdown"].min()
mdd = mdd.round(4) * -100

mdd_x = sp500.iloc[sp500["max_drawdown"].idxmin()]["date"]
mdd_y = sp500.iloc[sp500["max_drawdown"].idxmin()]["close"]
fig.add_annotation(x=mdd_x, y=mdd_y, text=f"Max Drawdown {mdd}%", font=dict(color="black",size=20), arrowcolor="red", arrowsize=3, arrowwidth=1, arrowhead=1, showarrow=True, xref="x", yref="y", ax=70, ay=240)


fig.update_layout(legend=dict(x=0.01, y=0.97, font=dict( family="Courier", size=16, color="black" ), bgcolor="LightSteelBlue", bordercolor="Black", borderwidth=1))

fig.update_layout(height=1200, plot_bgcolor='white')
fig.update_xaxes(mirror=True, ticks='outside', showline=True, linecolor='black', gridcolor='lightgrey')
fig.update_yaxes(mirror=True, ticks='outside', showline=True, linecolor='black', gridcolor='lightgrey')
fig.update_layout(title={'text': "S&P 500", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top',
                         'font': {'color':"black", 'size': 33}, 'subtitle': { 'text': f"ROI {roi}%", 'font': {'color':"gray", 'size': 19}}})
fig.show()b

,date,adj close,close,high,low,open,volume
174,2023-05-23,170.597244,172.669998,177.979996,171.399994,177.880005,2301700.0
412,2024-05-03,166.673553,167.070007,171.690002,166.949997,170.449997,3007000.0
445,2024-06-21,170.684006,171.089996,171.789993,169.350006,169.380005,4899700.0


---

# Trading Performance Metrics

#### 1. Max Drawdown (MDD)
Max Drawdown (MDD) should be as minimal as possible.  
MDD is a common metric for hedge funds and other trading shops to measure a trader's performance.   
MDD measures the percentage difference from the historical peak to the forward minimum.   
The plots below should elucidate the concept.  

#### 2. Return on Investment Percent (ROI)
 We are searching for a trading strategy that maximizes ROI with minimal and reasonable risk. 

### Optimized Moving Average 
#### <i>buy</i> when price below moving average
#### <i>sell</i> when price is above moving average
We look the optimize the size of the sliding window that captures the markets appetite of the S&P500 equities. This is how we profit from temporary spikes and lulls before the stock regresses to the theoretical mean.

### Trading Strategy Parameters (2)
#### 1. Window Size 
The number of previous prices to use for the moving average
#### 2. Performance Period
The amount of time we will use to backtest the trade execution of the strategy.
  
For each strategy we have an overall period (performance period) for which we trade. A typical year in the US include 252 trading days after weekends and holidays.  We will also use a single day as a single "tick" or "candle" for trading strategies that involve buying or selling based on a trading signal.

In [7]:
import itertools as it 
import pandas as pd

perf_periods = [int(252/4), int(252/2), 252, int(252*1.5)]
window_sizes = list(range(5,101,5))

perf_period_window_size_combos =  [{"perf_period":tpl[0], "window_size":tpl[1]} for tpl in it.product(perf_periods,window_sizes) if tpl[0] > tpl[1]]

print(len(perf_periods), "performance periods")
print(len(window_sizes), "windows sizes")

df_orig = pd.read_csv("sp500_stocks.csv", parse_dates=["Date"])
print(len(df_orig["Symbol"].unique()), "stock tickers")
print(len(perf_period_window_size_combos) * len(df_orig["Symbol"].unique()), "total backtests")

4 performance periods
20 windows sizes
503 stock tickers
36216 total backtests


---

# Vectorized Backtesting

##### Various trading performance performance periods (perf_periods) and window sizes for backtesting.

The following function 
* (1) Loads S&P500 data
* (2) Calculates the Simple Moving Average (SMA)
* (3) Generates Buy/Sell Signals from SMA
* (4) Executes Trades using Signals
* (5) Determines P&L
* (6) Calculates the ROI & MDD

In [8]:
def vectorized_sma_backtests(perf_period, win_sz, ticker=None):

    # (1) LOAD & TRANSFORM DATA
    df_orig = pd.read_csv("sp500_stocks.csv", parse_dates=["Date"])

    if ticker is not None:
        df_orig = df_orig[df_orig["Symbol"] == ticker]
    df_orig = df_orig.dropna()
    df_orig = df_orig[["Date", "Symbol", "Open", "Low", "High", "Close"]]
    df_orig.columns = list(map(str.lower,df_orig.columns))
    df_orig = df_orig.sort_values(["symbol", "date"])
    df_orig = df_orig.reset_index(drop=True)
            
    stock_dates = df_orig["date"].unique()
    stock_dates = stock_dates[stock_dates.argsort()][-perf_period:]

    df = df_orig.copy()
    df = df[df["date"] >= stock_dates.min()].reset_index(drop=True)    

    # (2) CALCULATE MOVING AVERAGE w/ WINDOW SIZE
    df["ma"] = df.groupby("symbol")["close"].rolling(window=win_sz, min_periods=0).mean().reset_index(0,drop=True)

    # (3) GENERATE BUY & SELL SIGNALS
    df["signal"] =  ( (df["close"] < df["ma"]).astype(int)* -1 + (df["close"] > df["ma"]).astype(int)  ) 
    df["signal"]  = (df["signal"] != df["signal"].shift(1)).astype(int) * df["signal"]
    df.loc[0, "signal"] = 0
    
    last_buy_price = df.loc[df["signal"] == -1].copy()["close"].rename("last_buy_price")
    df = df.merge(last_buy_price, left_index=True, right_index=True, how="left").fillna(0)
    df["last_buy_price"] = df["last_buy_price"].replace(0.0, np.nan).ffill().fillna(0)
    
    sell_mask = (df["signal"] == 1) & (df["last_buy_price"] > 0)

    # (4) EXECUTE TRADES
    df.loc[sell_mask, "profit_pct_plus_1"] = 1 + ( (df["close"] - df["last_buy_price"]) / df["last_buy_price"] )
    df["profit_pct_plus_1"] = df["profit_pct_plus_1"].fillna(1.0)

    # (5) DETERMINE P&L
    df["pct_cumprod"] = df.groupby("symbol")["profit_pct_plus_1"].cumprod()
    df["pct_cummax"] = df.groupby("symbol")["pct_cumprod"].cummax()
    
    df["drawdown"] = df["pct_cumprod"] - df["pct_cummax"]
    df["max_drawdown"] = df.groupby("symbol")["drawdown"].cummin()

    # (6) CALCULATE ROI & MDD
    roi =  df.groupby("symbol")["pct_cumprod"].last() - 1.0
    mdd = df.groupby("symbol")["max_drawdown"].min()

    result = dict(perf_period=perf_period, win_sz=win_sz, roi_min=roi.min(), roi_mean=roi.mean(), roi_max=roi.max(), mdd_min=mdd.min(), mdd_mean=mdd.mean())
    return result, df

---

# The Entire Standard & Poors 500 - <i><u>NO RAPIDS</u></i> --> <span style="color:red"><b> multiple minutes </b></span> to run backtests :(

##### Backtest various Moving Averages for All Stocks in the S&P500
##### +40K backtests 

In [9]:
!nproc

28


In [10]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            56Gi       1.3Gi        38Gi       2.0Mi        17Gi        55Gi
Swap:             0B          0B          0B


In [11]:
from tqdm import tqdm
import pandas as pd
import numpy as np

results_list = []
progress = tqdm(perf_period_window_size_combos)
for perf_period_window_size_combo in progress:    
    perf_period = perf_period_window_size_combo["perf_period"]
    win_sz = perf_period_window_size_combo["window_size"]
    progress.set_description(f"pp: {perf_period} win_sz: {win_sz} ")
    
    result, _ = vectorized_sma_backtests(perf_period=perf_period, win_sz=win_sz)
    results_list.append(result)

pp: 378 win_sz: 100 : 100%|██████████| 72/72 [02:49<00:00,  2.36s/it]


### Nearly 3 Minutes

---

# [RAPIDS](https://developer.nvidia.com/rapids)
##### Time is Money,  Accelerate Your Backtesting.
We need high performace computing to test as many strategies as possible across as many equities as possible.  To that end, we will use RAPIDS to ensure constant testing of your strategies.
Follow the RAPIDS Installation guide to install the correction version for your system:  
https://docs.rapids.ai/install#selector

![RAPIDS_selector](./RAPIDS_selector.png)

---

# The Entire Standard & Poors 500 - <i><u>w/ RAPIDS<u/></i> --> <span style="color:green"> <b> seconds </b></span> to run backtest :)

##### Backtest various Moving Averages for All Stocks in the S&P500

##### <b>RAPIDS</b> give us over <b>700%</b> boost in backtesting speed performance
  
  
### No Code Changes Except loading cudf.pandas

In [12]:
## load cudf.pandas kernel
%load_ext cudf.pandas

In [14]:
from tqdm import tqdm
import pandas as pd
import numpy as np


results_list = []
progress = tqdm(perf_period_window_size_combos)
for perf_period_window_size_combo in progress:    
    perf_period = perf_period_window_size_combo["perf_period"]
    win_sz = perf_period_window_size_combo["window_size"]
    progress.set_description(f"pp: {perf_period} win_sz: {win_sz} ")
    
    result, _ = vectorized_sma_backtests(perf_period=perf_period, win_sz=win_sz)
    results_list.append(result)

pp: 378 win_sz: 100 : 100%|██████████| 72/72 [00:24<00:00,  2.96it/s]


### 20 to 30 seconds

In [32]:
(120 + 49) / 24

7.041666666666667

### 700%+ Improvement

---

# Trading Performance Analysis  

### Plot Generator Function

In [20]:
def pairwise(iterable):
    # pairwise('ABCDEFG') → AB BC CD DE EF FG
    iterator = iter(iterable)
    a = next(iterator, None)
    for b in iterator:
        yield a, b
        a = b

In [21]:
def generate_mdd_roi_traces(results, perf_period_param):
    results_summary = results.query(f" perf_period == {perf_period_param} ")

    x = results_summary["win_sz"]
    y = results_summary["roi_mean"]*100
    
    x_pairs = pairwise(x)
    y_pairs = pairwise(y)
    roi_colors=['red' if any([i < roi_cutoff for i in y_values]) else '#1f77b4' for y_values in pairwise(y)]
        
    roi_trace = go.Scatter(x=results_summary["win_sz"], y=results_summary["mdd_mean"]*-100, 
                             name="MDD (redness ~ losses)", fill='tozeroy', mode='none',
                             fillgradient=dict(type="vertical", colorscale=[(0.0, "white"),(0.95, "red"), (1.0, "red")], ))
    
    mdd_trace = go.Scatter(x=x, y=y,  showlegend=True, name="ROI", mode='lines', opacity=0.7, line={'color': "black",  'width':4})
    
    # piecemeal ROI plot to show red for non-positive ROIs
    piecemeal_traces = []
    for x, y, color in zip(x_pairs, y_pairs, roi_colors):
        piecemeal_traces.append(go.Scatter(x=x, y=y, showlegend=False, mode='lines', opacity=.6, line={'color': color,  'width':0.95}))
    
    return roi_trace, mdd_trace, piecemeal_traces

### Tabular Results

In [22]:
results = pd.DataFrame(results_list)
results_summary = results.sort_values(["mdd_min", "roi_mean"], ascending=True)
results_summary.head(5)

,perf_period,win_sz,roi_min,roi_mean,roi_max,mdd_min,mdd_mean
53,378,10,-0.997096,1.142155,63.062714,-16.222246,-0.576481
57,378,30,-0.996892,1.185733,62.307560,-15.818044,-0.503630
55,378,20,-0.997043,1.215337,70.805449,-14.717758,-0.533444
56,378,25,-0.996940,1.208964,65.637189,-14.659468,-0.515434
58,378,35,-0.996781,1.159912,60.950255,-14.659412,-0.491891


### Visual Results

In [23]:
from plotly.subplots import make_subplots

roi_cutoff  = 0.10

fig = make_subplots(rows=2, cols=2, shared_xaxes=False, shared_yaxes=True, 
                    vertical_spacing=0.15, 
                    subplot_titles=[f"Perf Period = {pp}" for pp in results.perf_period.unique()])

for plot_idx, perf_period in enumerate(results.perf_period.unique()):    
    roi_trace, mdd_trace, piecemeal_traces = generate_mdd_roi_traces(results=results, perf_period_param=perf_period)    
    col = plot_idx % 2 + 1
    row = int(plot_idx > 1) + 1       
    fig.add_trace(roi_trace,  row=row, col=col)
    fig.add_trace(mdd_trace,  row=row, col=col)
    for piecemeal_trace in piecemeal_traces:
        fig.add_trace(piecemeal_trace,  row=row, col=col)
    fig.update_xaxes(title_text="window size", row=row, col=col)
    fig.update_yaxes(title_text="ROI% & MDD%", row=row, col=col)


fig.update_layout(showlegend=False, height=1000)
fig.update_layout(plot_bgcolor='white')
fig.update_xaxes(mirror=True, ticks='outside', showline=True, linecolor='black', gridcolor='lightgrey')
fig.update_yaxes(mirror=True, ticks='outside', showline=True, linecolor='black', gridcolor='lightgrey')

fig.show()

---

# Sample Stock using Backtesting Results
Top company at time of writing is AAPL (Apple)

In [24]:
pd.read_csv("sp500_companies.csv").sort_values("Marketcap", ascending=False).head(1)

,Exchange,Symbol,Shortname,Longname,Sector,Industry,Currentprice,Marketcap,Ebitda,Revenuegrowth,City,State,Country,Fulltimeemployees,Longbusinesssummary,Weight
0,NMS,AAPL,Apple Inc.,Apple Inc.,Technology,Consumer Electronics,222.5,3382912221184,131781001216,0.049,Cupertino,CA,United States,161000,"Apple Inc. designs, manufactures, and markets ...",0.064793


### AAPL Sample Data

In [26]:
df = pd.read_csv("sp500_stocks.csv")
aapl = df.query("Symbol == 'AAPL' ")
del df
aapl.tail(3).round(2)

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
147957,2024-09-11,AAPL,222.66,222.66,223.09,217.89,221.46,44587100.0
147958,2024-09-12,AAPL,222.77,222.77,223.55,219.82,222.50,37498200.0
147959,2024-09-13,AAPL,222.50,222.50,224.04,221.91,223.58,36722900.0


### Backtest AAPL with various Moving Averages

In [27]:
from tqdm import tqdm
aapl_results_list = []
progress = tqdm(perf_period_window_size_combos)

for perf_period_window_size_combo in progress:
    
    perf_period = perf_period_window_size_combo["perf_period"]
    win_sz = perf_period_window_size_combo["window_size"]
    progress.set_description(f"pp: {perf_period} win_sz: {win_sz} ")
    
    result, _ = vectorized_sma_backtests(perf_period=perf_period, win_sz=win_sz, ticker='AAPL')
    aapl_results_list.append(result)

pp: 378 win_sz: 100 : 100%|██████████| 72/72 [00:12<00:00,  5.87it/s]


### Tabular Backtesting Results

In [28]:
aapl_results = pd.DataFrame(aapl_results_list)
aapl_results.sort_values(["mdd_min", "roi_max"], ascending=[False, False]).head(3)

,perf_period,win_sz,roi_min,roi_mean,roi_max,mdd_min,mdd_mean
17,126,30,0.140590,0.140590,0.140590,0.0,0.0
20,126,45,0.135784,0.135784,0.135784,0.0,0.0
18,126,35,0.124623,0.124623,0.124623,0.0,0.0


### Visual Backtesting Results

In [29]:
fig = make_subplots(rows=2, cols=2, shared_xaxes=False, shared_yaxes=True, 
                    vertical_spacing=0.15, subplot_titles=[f"Perf Period = {pp}" for pp in aapl_results.perf_period.unique()])

for plot_idx, perf_period in enumerate(aapl_results.perf_period.unique()):    
    roi_trace, mdd_trace, piecemeal_traces = generate_mdd_roi_traces(results=aapl_results, perf_period_param=perf_period)    
    col = plot_idx % 2 + 1
    row = int(plot_idx > 1) + 1       
    fig.add_trace(roi_trace,  row=row, col=col)
    fig.add_trace(mdd_trace,  row=row, col=col)
    for piecemeal_trace in piecemeal_traces:
        fig.add_trace(piecemeal_trace,  row=row, col=col)
    fig.update_xaxes(title_text="window size", row=row, col=col)
    fig.update_yaxes(title_text="ROI% & MDD%", row=row, col=col)

fig.update_layout(showlegend=False, height=1000)
fig.update_layout(plot_bgcolor='white')
fig.update_xaxes(mirror=True, ticks='outside', showline=True, linecolor='black', gridcolor='lightgrey')
fig.update_yaxes(mirror=True, ticks='outside', showline=True, linecolor='black', gridcolor='lightgrey')

fig.show()

### Trade Execution with Best Parameters

In [30]:
perf_period_window_size_combo = aapl_results.sort_values(["mdd_min", "roi_max"], ascending=[False, False]).head(1)[["perf_period", "win_sz"]].to_dict("records")[0]
print(perf_period_window_size_combo)

perf_period=perf_period_window_size_combo["perf_period"]
win_sz=perf_period_window_size_combo["win_sz"]

marker_size = 10
marker_border = 1.0

result, df = vectorized_sma_backtests(perf_period=perf_period, win_sz=win_sz, ticker="AAPL")

stock_plot = go.Ohlc(x=df['date'], open=df['open'], high=df['high'], low=df['low'], close=df['close'], name="AAPL", increasing_line_color= 'cyan', decreasing_line_color= 'gray')

fig = go.Figure(data=[stock_plot])

# MA
fig.add_trace(go.Scatter(x=df["date"], y=df["ma"], name=f"MA {win_sz}", opacity=0.3, line=dict(width=2, color="blue")))

# BUY
buy_mask = df["signal"] == -1
purchases = df.loc[buy_mask].copy().reset_index(drop=True)
fig.add_trace(go.Scatter(
    x=purchases["date"], y=purchases["close"], mode="markers", name="buy", 
    marker=dict(symbol="star-triangle-up", size=marker_size, color='blue', line=dict(width=marker_border, color="DarkSlateGrey"))))

# SELL
sales = df.loc[df["signal"] == 1].copy().reset_index(drop=True)
fig.add_trace(go.Scatter(
    x=sales["date"], y=sales["close"], mode="markers", name="sell",
    marker=dict(symbol="star-triangle-down", size=marker_size,  color='green', line=dict(width=marker_border, color="DarkSlateGrey"))))


fig.update_layout(legend=dict(x=0.01, y=0.97, font=dict( family="Courier", size=16, color="black" ), bgcolor="LightSteelBlue", bordercolor="Black", borderwidth=1))

fig.update_layout(height=900)
fig.update_layout(plot_bgcolor='white')
fig.update_xaxes(mirror=True, ticks='outside', showline=True, linecolor='black', gridcolor='lightgrey')
fig.update_yaxes(mirror=True, ticks='outside', showline=True, linecolor='black', gridcolor='lightgrey')

roi = df.iloc[-1]["pct_cumprod"] - 1
roi = roi.round(4) * 100

fig.update_layout(title={'text': "AAPL", 'y':0.85, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top',
                         'font': {'color':"black", 'size': 33}, 'subtitle': { 'text': f"ROI {roi}%", 'font': {'color':"gray", 'size': 19}}})
fig.show()

{'perf_period': 126, 'win_sz': 30}
